Importamos las librerías necesarias 

In [41]:
import os
import requests
import numpy as np
import pandas as pd
import json
from dotenv import load_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

Se establece la conexión a la api para la ingesta de datos

In [42]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv(dotenv_path='.env.mongo_credentials')

# Obtener las credenciales desde las variables de entorno
url_contacts = "https://unify.kolla.dev/dental/v1/contacts"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {os.getenv('BEARER_TOKEN')}",
    "connector-id": os.getenv('CONNECTOR_ID'),
    "consumer-id": os.getenv('CONSUMER_ID')
}

# Realizar la solicitud GET
try:
    response = requests.get(url_contacts, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()

        # Convertir los datos a un DataFrame de pandas
        df_contacts = pd.DataFrame(data)  # Asegúrate de que los datos sean una lista de diccionarios

        print("Datos obtenidos y convertidos a DataFrame:")
        print(df_contacts.head())  # Muestra las primeras filas del DataFrame
    else:
        print(f"Error al obtener los datos: {response.status_code}")
        print("Detalle:", response.text)

except Exception as e:
    print(f"Error al conectar con la API: {e}")


Datos obtenidos y convertidos a DataFrame:
                                            contacts  \
0  {'name': 'contacts/807', 'remote_id': '807', '...   
1  {'name': 'contacts/806', 'remote_id': '806', '...   
2  {'name': 'contacts/805', 'remote_id': '805', '...   
3  {'name': 'contacts/804', 'remote_id': '804', '...   
4  {'name': 'contacts/803', 'remote_id': '803', '...   

                                     next_page_token  
0  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
1  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
2  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
3  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
4  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  


In [43]:
df_contacts.head()

,contacts,next_page_token
0,"{'name': 'contacts/807', 'remote_id': '807', '...",NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...
1,"{'name': 'contacts/806', 'remote_id': '806', '...",NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...
2,"{'name': 'contacts/805', 'remote_id': '805', '...",NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...
3,"{'name': 'contacts/804', 'remote_id': '804', '...",NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...
4,"{'name': 'contacts/803', 'remote_id': '803', '...",NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...


In [44]:
# Desanidar la columna "contact_info"
contact_info_df = pd.json_normalize(df_contacts['contacts'])

# Combinar con el DataFrame original (sin duplicar la columna desanidada)
df_contacts = pd.concat([df_contacts.drop(columns=['contacts']), contact_info_df], axis=1)

# Mostrar el DataFrame resultante
df_contacts.head()

,next_page_token,name,remote_id,type,given_name,family_name,preferred_name,gender,birth_date,notes,...,state,preferred_provider,first_visit,guarantor,opt_ins,create_time,update_time,additional_data.ImageFolder,opt_ins.sms,opt_ins.email
0,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,contacts/807,807,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,...,ACTIVE,None,0001-01-01,contacts/807,NaN,None,2025-01-19T16:15:13Z,mahermahmoud807,NaN,NaN
1,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,contacts/806,806,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,...,ACTIVE,None,0001-01-01,contacts/806,NaN,None,2025-01-19T15:32:05Z,mahermahmoud806,NaN,NaN
2,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,contacts/805,805,PATIENT,Jorge,Martínez,,GENDER_OTHER,0000-00-00,,...,ACTIVE,None,0001-01-01,contacts/805,NaN,None,2025-01-18T15:43:02Z,martnezjorge805,NaN,NaN
3,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,contacts/804,804,PATIENT,Jorge,Martinez,,GENDER_OTHER,0000-00-00,,...,ACTIVE,None,0001-01-01,contacts/804,NaN,None,2025-01-18T15:01:53Z,martinezjorge804,NaN,NaN
4,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,contacts/803,803,PATIENT,Javier,Perez,,GENDER_OTHER,0000-00-00,,...,ACTIVE,None,0001-01-01,contacts/803,NaN,None,2025-01-17T23:58:13Z,perezjavier803,NaN,NaN


In [45]:
# Nombre de la columna a eliminar
columna_a_eliminar = 'next_page_token'

# Eliminar la columna directamente del DataFrame original
df_contacts.drop(columns=[columna_a_eliminar], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_contacts.head()


,name,remote_id,type,given_name,family_name,preferred_name,gender,birth_date,notes,addresses,...,state,preferred_provider,first_visit,guarantor,opt_ins,create_time,update_time,additional_data.ImageFolder,opt_ins.sms,opt_ins.email
0,contacts/807,807,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,[],...,ACTIVE,None,0001-01-01,contacts/807,NaN,None,2025-01-19T16:15:13Z,mahermahmoud807,NaN,NaN
1,contacts/806,806,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,[],...,ACTIVE,None,0001-01-01,contacts/806,NaN,None,2025-01-19T15:32:05Z,mahermahmoud806,NaN,NaN
2,contacts/805,805,PATIENT,Jorge,Martínez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/805,NaN,None,2025-01-18T15:43:02Z,martnezjorge805,NaN,NaN
3,contacts/804,804,PATIENT,Jorge,Martinez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/804,NaN,None,2025-01-18T15:01:53Z,martinezjorge804,NaN,NaN
4,contacts/803,803,PATIENT,Javier,Perez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/803,NaN,None,2025-01-17T23:58:13Z,perezjavier803,NaN,NaN


In [46]:
# Obtener las credenciales desde las variables de entorno
url_appointments = "https://unify.kolla.dev/dental/v1/appointments"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {os.getenv('BEARER_TOKEN')}",
    "connector-id": os.getenv('CONNECTOR_ID'),
    "consumer-id": os.getenv('CONSUMER_ID')
}

# Realizar la solicitud GET
try:
    response = requests.get(url_appointments, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()

        # Convertir los datos a un DataFrame de pandas
        df_appointments = pd.DataFrame(data)  # Asegúrate de que los datos sean una lista de diccionarios

        print("Datos obtenidos y convertidos a DataFrame:")
        print(df_appointments.head())  # Muestra las primeras filas del DataFrame
    else:
        print(f"Error al obtener los datos: {response.status_code}")
        print("Detalle:", response.text)

except Exception as e:
    print(f"Error al conectar con la API: {e}")

Datos obtenidos y convertidos a DataFrame:
                                        appointments  \
0  {'name': 'appointments/1358', 'remote_id': '13...   
1  {'name': 'appointments/1356', 'remote_id': '13...   
2  {'name': 'appointments/1355', 'remote_id': '13...   
3  {'name': 'appointments/860', 'remote_id': '860...   
4  {'name': 'appointments/734', 'remote_id': '734...   

                                     next_page_token  
0  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
1  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
2  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
3  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  
4  NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...  


In [47]:
# Desanidar la columna "appointments"
appointments_info_df = pd.json_normalize(df_appointments['appointments'])

# Combinar con el DataFrame original (sin duplicar la columna desanidada)
df_appointments = pd.concat([df_appointments.drop(columns=['appointments']), appointments_info_df], axis=1)

# Mostrar el DataFrame resultante
df_appointments.head()

,next_page_token,name,remote_id,contact_id,location,start_time,end_time,wall_start_time,wall_end_time,time_zone,...,contact.name,contact.remote_id,contact.given_name,contact.family_name,scheduler.name,scheduler.remote_id,scheduler.type,scheduler.display_name,additional_data.new_patient,scheduler
0,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,appointments/1358,1358,contacts/16,,2025-01-16T17:00:00Z,2025-01-16T17:30:00Z,2025-01-16 10:00:00,2025-01-16 10:30:00,,...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
1,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,appointments/1356,1356,contacts/16,,2025-01-15T20:10:00Z,2025-01-15T20:40:00Z,2025-01-15 13:10:00,2025-01-15 13:40:00,,...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
2,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,appointments/1355,1355,contacts/16,,2025-01-14T20:10:00Z,2025-01-14T20:40:00Z,2025-01-14 13:10:00,2025-01-14 13:40:00,,...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
3,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,appointments/860,860,contacts/583,,2024-11-05T17:20:00Z,2024-11-05T17:50:00Z,2024-11-05 10:20:00,2024-11-05 10:50:00,,...,,583,Junior,Berry,,user_1,staff,,true,NaN
4,NX8DAQEJUGFnZVRva2VuAf-AAAECAQZPZmZzZXQBBAABD1...,appointments/734,734,contacts/22,,2024-06-26T19:00:00Z,2024-06-26T19:40:00Z,2024-06-26 13:00:00,2024-06-26 13:40:00,,...,,22,Clinton,Berry,,user_1,staff,,NaN,NaN


In [48]:
# Nombre de la columna a eliminar
columna_a_eliminar = 'next_page_token'

# Eliminar la columna directamente del DataFrame original
df_appointments.drop(columns=[columna_a_eliminar], inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_appointments.head()

,name,remote_id,contact_id,location,start_time,end_time,wall_start_time,wall_end_time,time_zone,providers,...,contact.name,contact.remote_id,contact.given_name,contact.family_name,scheduler.name,scheduler.remote_id,scheduler.type,scheduler.display_name,additional_data.new_patient,scheduler
0,appointments/1358,1358,contacts/16,,2025-01-16T17:00:00Z,2025-01-16T17:30:00Z,2025-01-16 10:00:00,2025-01-16 10:30:00,,"[{'name': 'resources/provider_1', 'remote_id':...",...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
1,appointments/1356,1356,contacts/16,,2025-01-15T20:10:00Z,2025-01-15T20:40:00Z,2025-01-15 13:10:00,2025-01-15 13:40:00,,"[{'name': 'resources/provider_3', 'remote_id':...",...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
2,appointments/1355,1355,contacts/16,,2025-01-14T20:10:00Z,2025-01-14T20:40:00Z,2025-01-14 13:10:00,2025-01-14 13:40:00,,"[{'name': 'resources/provider_3', 'remote_id':...",...,,16,Jane,Smith,,user_1,staff,,NaN,NaN
3,appointments/860,860,contacts/583,,2024-11-05T17:20:00Z,2024-11-05T17:50:00Z,2024-11-05 10:20:00,2024-11-05 10:50:00,,"[{'name': 'resources/provider_3', 'remote_id':...",...,,583,Junior,Berry,,user_1,staff,,true,NaN
4,appointments/734,734,contacts/22,,2024-06-26T19:00:00Z,2024-06-26T19:40:00Z,2024-06-26 13:00:00,2024-06-26 13:40:00,,"[{'name': 'resources/provider_1', 'remote_id':...",...,,22,Clinton,Berry,,user_1,staff,,NaN,NaN


In [49]:
# Obtener las credenciales desde las variables de entorno
url_treatment_plans = "https://unify.kolla.dev/dental/v1/treatmentPlans"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {os.getenv('BEARER_TOKEN')}",
    "connector-id": os.getenv('CONNECTOR_ID'),
    "consumer-id": os.getenv('CONSUMER_ID')
}

# Realizar la solicitud GET
try:
    response = requests.get(url_treatment_plans, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()

        # Convertir los datos a un DataFrame de pandas
        df_treatment_plans = pd.DataFrame(data)  # Asegúrate de que los datos sean una lista de diccionarios

        print("Datos obtenidos y convertidos a DataFrame:")
        print(df_treatment_plans.head())  # Muestra las primeras filas del DataFrame
    else:
        print(f"Error al obtener los datos: {response.status_code}")
        print("Detalle:", response.text)

except Exception as e:
    print(f"Error al conectar con la API: {e}")

Datos obtenidos y convertidos a DataFrame:
                                     treatment_plans next_page_token
0  {'name': 'treatmentPlan/5', 'remote_id': '5', ...                
1  {'name': 'treatmentPlan/4', 'remote_id': '4', ...                
2  {'name': 'treatmentPlan/3', 'remote_id': '3', ...                
3  {'name': 'treatmentPlan/2', 'remote_id': '2', ...                
4  {'name': 'treatmentPlan/1', 'remote_id': '1', ...                


In [50]:
# Desanidar la columna "treatment_plans"
treatment_plans_info = pd.json_normalize(df_treatment_plans['treatment_plans'])

# Combinar con el DataFrame original (sin duplicar la columna desanidada)
df_treatment_plans = pd.concat([df_treatment_plans.drop(columns=['treatment_plans']), treatment_plans_info], axis=1)

# Mostrar el DataFrame resultante
df_treatment_plans.head()


,next_page_token,name,remote_id,state,contact,create_time,update_time
0,,treatmentPlan/5,5,STATE_UNSPECIFIED,contacts/8,None,2024-03-29T13:59:31Z
1,,treatmentPlan/4,4,STATE_UNSPECIFIED,contacts/9,None,2023-06-28T15:55:32Z
2,,treatmentPlan/3,3,STATE_UNSPECIFIED,contacts/10,None,2022-01-31T14:28:43Z
3,,treatmentPlan/2,2,STATE_UNSPECIFIED,contacts/11,None,2021-05-17T17:46:20Z
4,,treatmentPlan/1,1,STATE_UNSPECIFIED,contacts/15,None,2017-08-31T10:33:37Z


In [51]:
# Nombre de la columna a eliminar
columna_a_eliminar = 'next_page_token'

# Eliminar la columna directamente del DataFrame original
df_treatment_plans.drop(columns=[columna_a_eliminar], inplace=True)

# Reiniciar el índice del DataFrame 
df_treatment_plans.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_treatment_plans.head()

,name,remote_id,state,contact,create_time,update_time
0,treatmentPlan/5,5,STATE_UNSPECIFIED,contacts/8,None,2024-03-29T13:59:31Z
1,treatmentPlan/4,4,STATE_UNSPECIFIED,contacts/9,None,2023-06-28T15:55:32Z
2,treatmentPlan/3,3,STATE_UNSPECIFIED,contacts/10,None,2022-01-31T14:28:43Z
3,treatmentPlan/2,2,STATE_UNSPECIFIED,contacts/11,None,2021-05-17T17:46:20Z
4,treatmentPlan/1,1,STATE_UNSPECIFIED,contacts/15,None,2017-08-31T10:33:37Z


In [52]:
# Obtener las credenciales desde las variables de entorno
url_transactions = "https://unify.kolla.dev/dental/v1/transactions"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {os.getenv('BEARER_TOKEN')}",
    "connector-id": os.getenv('CONNECTOR_ID'),
    "consumer-id": os.getenv('CONSUMER_ID')
}

# Realizar la solicitud GET
try:
    response = requests.get(url_transactions, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a JSON
        data = response.json()

        # Convertir los datos a un DataFrame de pandas
        df_transactions = pd.DataFrame(data)  # Asegúrate de que los datos sean una lista de diccionarios

        print("Datos obtenidos y convertidos a DataFrame:")
        print(df_transactions.head())  # Muestra las primeras filas del DataFrame
    else:
        print(f"Error al obtener los datos: {response.status_code}")
        print("Detalle:", response.text)

except Exception as e:
    print(f"Error al conectar con la API: {e}")

Datos obtenidos y convertidos a DataFrame:
                                        transactions next_page_token
0  {'name': 'transactions/proc-142', 'remote_id':...                
1  {'name': 'transactions/proc-134', 'remote_id':...                
2  {'name': 'transactions/proc-133', 'remote_id':...                
3  {'name': 'transactions/proc-131', 'remote_id':...                
4  {'name': 'transactions/proc-130', 'remote_id':...                


In [53]:
# Desanidar la columna "treatment_plans"
transactions_info = pd.json_normalize(df_transactions['transactions'])

# Combinar con el DataFrame original (sin duplicar la columna desanidada)
df_transactions = pd.concat([df_transactions.drop(columns=['transactions']), transactions_info], axis=1)

# Mostrar el DataFrame resultante
df_transactions.head()

,next_page_token,name,remote_id,contact,guarantor,provider,amount,note,claim,transaction_type,transaction_time,procedure_code,procedure_description,procedure_time,transaction_entry_time,create_time,update_time
0,,transactions/proc-142,proc-142,contacts/16,,resources/provider_1,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:18:25Z
1,,transactions/proc-134,proc-134,contacts/16,,resources/provider_3,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:11:07Z
2,,transactions/proc-133,proc-133,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:04:05Z
3,,transactions/proc-131,proc-131,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T15:50:16Z
4,,transactions/proc-130,proc-130,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T15:50:05Z


In [54]:
# Nombre de la columna a eliminar
columna_a_eliminar = 'next_page_token'

# Eliminar la columna directamente del DataFrame original
df_transactions.drop(columns=[columna_a_eliminar], inplace=True)

# Reiniciar el índice del DataFrame 
df_transactions.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del DataFrame actualizado
df_transactions.head()

,name,remote_id,contact,guarantor,provider,amount,note,claim,transaction_type,transaction_time,procedure_code,procedure_description,procedure_time,transaction_entry_time,create_time,update_time
0,transactions/proc-142,proc-142,contacts/16,,resources/provider_1,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:18:25Z
1,transactions/proc-134,proc-134,contacts/16,,resources/provider_3,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:11:07Z
2,transactions/proc-133,proc-133,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T16:04:05Z
3,transactions/proc-131,proc-131,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T15:50:16Z
4,transactions/proc-130,proc-130,contacts/15,,,0,,,PROCEDURE,2025-01-14T00:00:00Z,D9986,,None,2025-01-14T00:00:00Z,None,2025-01-14T15:50:05Z


Cálculo de métricas

Unscheduled Opportunity

In [72]:
# Filtrar pacientes no programados
unscheduled_patients = df_contacts[~df_contacts['remote_id'].isin(df_appointments['contact.remote_id'])]

# Producción promedio por visita de higiene
average_hygiene_production = df_transactions[df_transactions['procedure_code'] == 'T6357']['amount'].mean()

# Calcular Unscheduled Opportunity
unscheduled_opportunity = len(unscheduled_patients) * average_hygiene_production

In [76]:
unscheduled_patients.head()

,name,remote_id,type,given_name,family_name,preferred_name,gender,birth_date,notes,addresses,...,state,preferred_provider,first_visit,guarantor,opt_ins,create_time,update_time,additional_data.ImageFolder,opt_ins.sms,opt_ins.email
2,contacts/805,805,PATIENT,Jorge,Martínez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/805,NaN,None,2025-01-18T15:43:02Z,martnezjorge805,NaN,NaN
3,contacts/804,804,PATIENT,Jorge,Martinez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/804,NaN,None,2025-01-18T15:01:53Z,martinezjorge804,NaN,NaN
4,contacts/803,803,PATIENT,Javier,Perez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/803,NaN,None,2025-01-17T23:58:13Z,perezjavier803,NaN,NaN
5,contacts/802,802,PATIENT,Luis,Valera,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/802,NaN,None,2025-01-17T23:36:28Z,valeraluis802,NaN,NaN
6,contacts/801,801,PATIENT,Rodrigo,Perez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/801,NaN,None,2025-01-17T23:36:07Z,perezrodrigo801,NaN,NaN


In [74]:
average_hygiene_production

103.125

In [73]:
unscheduled_opportunity

4743.75

Annual Opportunity Estimated

In [56]:
# Eliminar espacios y caracteres no deseados en la columna 'amount'
df_transactions['amount'] = df_transactions['amount'].str.strip()

# Convertir a tipo numérico (manteniendo negativos); reemplazar valores no convertibles por NaN
df_transactions['amount'] = pd.to_numeric(df_transactions['amount'], errors='coerce')

# Verificar si hay valores NaN después de la conversión
if df_transactions['amount'].isna().any():
    print("Advertencia: Hay valores no convertibles en 'amount'. Revisa estas filas:")
    print(df_transactions[df_transactions['amount'].isna()])


In [57]:
# Recolección promedio anual por paciente
average_annual_collection = df_transactions.groupby('contact')['amount'].sum().mean()

# Calcular Annual Opportunity Estimated
annual_opportunity_estimated = len(unscheduled_patients) * average_annual_collection


Pre Appointment

In [58]:
# Pacientes activos
active_patients = df_contacts[df_contacts['state'] == 'ACTIVE']

# Pacientes activos con citas
scheduled_patients = df_appointments[df_appointments['contact.remote_id'].isin(active_patients['remote_id'])]

# Calcular Pre Appointment (%)
pre_appointment = (len(scheduled_patients['contact.remote_id'].unique()) / len(active_patients)) * 100


Broken 

In [59]:
def determine_status(row):
    if row['confirmed']:
        return 'confirmed'
    elif row['cancelled']:
        return 'cancelled'
    elif row['completed']:
        return 'completed'
    elif row['broken']:
        return 'broken'
    else:
        return 'unknown'

# Crear la columna 'status' basada en las columnas booleanas
df_appointments['status'] = df_appointments.apply(determine_status, axis=1)

# Eliminar las columnas booleanas si ya no son necesarias
df_appointments = df_appointments.drop(['confirmed', 'cancelled', 'completed', 'broken'], axis=1)


In [60]:
# Filtrar citas canceladas o no cumplidas
broken_appointments = df_appointments[
    (df_appointments['status'] == 'cancelled') | (df_appointments['status'] == 'no-show')
]

# Calcular número de pacientes que han faltado o cancelado
broken_count = len(broken_appointments)


Production per visit

In [61]:
df_transactions['transaction_time'] = pd.to_datetime(df_transactions['transaction_time'], errors='coerce')


In [62]:
df_transactions['transaction_time'] = df_transactions['transaction_time'].dt.tz_localize(None)


In [63]:
from datetime import datetime, timedelta

# Filtrar transacciones de los últimos 60 días
last_60_days = datetime.now() - timedelta(days=60)
recent_transactions = df_transactions[
    pd.to_datetime(df_transactions['transaction_time']) >= last_60_days
]

# Calcular producción promedio por visita
production_per_visit = recent_transactions.groupby('contact')['amount'].sum().mean()


Production

In [64]:
# Número de visitas (citas completadas)
completed_appointments = df_appointments[df_appointments['status'] == 'completed']

# Calcular producción total
total_production = len(completed_appointments) * production_per_visit


Gross

In [65]:
# Calcular el monto total recolectado (gross)
gross = df_transactions['amount'].sum()


Adjustments

In [66]:
# Calcular ajustes (Write-off + Adjustments)
adjustments = df_transactions[df_transactions['transaction_type'].isin(['Write-off', 'Adjustment'])]['amount'].sum()


Net Profit

In [67]:
# Calcular la ganancia neta
net_profit = gross - adjustments


In [70]:
df_contacts.head()

,name,remote_id,type,given_name,family_name,preferred_name,gender,birth_date,notes,addresses,...,state,preferred_provider,first_visit,guarantor,opt_ins,create_time,update_time,additional_data.ImageFolder,opt_ins.sms,opt_ins.email
0,contacts/807,807,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,[],...,ACTIVE,None,0001-01-01,contacts/807,NaN,None,2025-01-19T16:15:13Z,mahermahmoud807,NaN,NaN
1,contacts/806,806,PATIENT,Mahmoud,Maher,,GENDER_OTHER,2000-02-24,,[],...,ACTIVE,None,0001-01-01,contacts/806,NaN,None,2025-01-19T15:32:05Z,mahermahmoud806,NaN,NaN
2,contacts/805,805,PATIENT,Jorge,Martínez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/805,NaN,None,2025-01-18T15:43:02Z,martnezjorge805,NaN,NaN
3,contacts/804,804,PATIENT,Jorge,Martinez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/804,NaN,None,2025-01-18T15:01:53Z,martinezjorge804,NaN,NaN
4,contacts/803,803,PATIENT,Javier,Perez,,GENDER_OTHER,0000-00-00,,[],...,ACTIVE,None,0001-01-01,contacts/803,NaN,None,2025-01-17T23:58:13Z,perezjavier803,NaN,NaN


In [ ]:
# Directorio donde se guardarán los archivos
directorio = './datasets_BI/'

# Exportar df_contacts
df_contacts.to_csv(directorio + 'df_contacts.csv', index=False)

# Exportar df_appointments
df_appointments.to_csv(directorio + 'df_appointments.csv', index=False)

# Exportar df_appointments
df_transactions.to_csv(directorio + 'df_transactions.csv', index=False)

print("Archivos CSV exportados correctamente.")